## Setting up AWS Athena


### Step 1: Go to AWS Glue Service to Create and Run a "Crawler"

* A crawler is an automated service in AWS that looks through a set of classifiers prioritized by the schema to create metadata tables.  

* To get the most recent version of the metadata, which is updated by SRA daily, it is necessary to rerun the crawler.

* This differs from BigQuery because BigQuery database and table names are static and the metadata tables are recreated daily by SRA using automated scripts.

<img src="./img/crawler.png" align="left" width="700"/>

### Step 2: Confirm the Name of the Database and the Tables

* When the crawler is created, the user sets the database name. In this workshop, "sra" was the name set by the crawler.

* The level of the tables that show up in the database depends on which datastore is targeted by the crawler. In this case, all the SRA tables are visible because the crawler was set to run on the highest level directory **s3://sra-pub-metadata-us-east-1**

* Note that the tables are all in one database called **"sra"**. This differs from BigQuery which has two sets of tables in its schema: **"sra"** and **"sra_tax_analysis_tool"**


#### Athena Tables

<img src="./img/table.png" align="center" width="700"/>





#### BigQuery Tables

<img src="./img/BigQuery.png" align="center" width="350"/>


The Query workspace for Athena looks very similar to the BigQuery workspace.  The syntax is almost identical, but before starting, make note of how the Database and Tables are structured. The column names are exactly the same between the two providers.

<img src="./img/athena.png" align="center" width="800"/>

### Step 3: Create a "Staging" Bucket

* Athena differs from BigQuery in that it requires a user to create a bucket to hold all the metadata and query outputs

* This is true for both command line interface as well as direct console queries.

### Run the following command to create your bucket for this workshop

This script also applies credentials that allow this notebook (which is on GCP) to access AWS to create buckets and run Athena. It also creates a connection using sqlalchemy to the Athena database.

*The script is unique to this workshop, so make sure to add your own AWS credentials when working outside of this event. Refer to the setup_AWS_credentials.ipynb notebook for tips and tricks on how to do this.*

In [ ]:
%run Setup_AWS_Credentials.py

### Test that the Credentials were Loaded Correctly

In [ ]:
%%sql

SELECT acc, bioproject 
FROM sra.metadata
WHERE organism = 'Homo sapiens'
LIMIT 5



# Challenge : To find RNA-Seq data from estrogen receptor negative and progesterone positive breast cancer cell lines.


Hormone receptor positive (HR+) breast cancer cells may have either estrogen receptors (ER+) or progesterone receptors (PR+) or both.

We have already found one search strategy to find a dataset that had ER+ breast cancer cell lines.

This case study will look for HR+ breast cancer cell lines that are ER- and PR+

This will requre combining search strategies to narrow our choices.

These searchers are taking place on Athena, so the syntax is mostly the same with some subtle differences.

Some of they syntax, such as capitalization or lowercase, is dicated by the tool we are using, so if a query in this notebook does not work exactly the same in Athena, try that first.

### Start with a more general search that uses an "or" to specify receptor types

In [ ]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')


### A lot of the metadata we are picking up is not related to samples. So the next search specifies that the key value needs to end in "sam" by using a wildcard "%" before "sam" and not after.

In [ ]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE '%sam'





### Adding the word "receptor" in front of the wildcard here may limit the samples to a reasonable set for the next steps. 

There are several strategies to find interesting data sets, this is one that seems to work just within this case study. A good practice would be to spend some time trying different methods to get the best data set for your research question.

In [ ]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE 'receptor%sam'





#### This looks like a good set, let's pull out the accession list for downstream analysis.

The syntax is slightly different because we are using sqlmagic - a different program than bigquery.

The output of a query can be saved as a results object which is easily made into a dataframe for downstream analysis.  We are creating a results object called "accessions" and then saving it into a text file called "cell_line_ER_HR.txt"



In [ ]:
%%sql accessions <<


SELECT acc
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE 'receptor%sam'

#### The output is different in SQL magic compared to BigQuery magic in the other notebook.

It is not a dataframe yet, it would need to have accessions.DataFrame() to show the index as in the BigQuery results object.

In [ ]:
accessions

In [ ]:
accessions.DataFrame()

In [ ]:
file = open("cell_line_ER_HR.txt", "w")
file.write(accessions.DataFrame().to_string(index=False, header=False))
file.close()

***

## Example Bioinformatic Workflow

***

### We may not have time to go over this in detail, the code is here for your information ###

<img src="./img/Workflow_demo.png" align="left" width="700"/>




### We are going to skip down to the final analysis for this demo  ###

In [ ]:
%%bash

cat cell_line_ER_HR.txt

### Configure SRA Toolkit

In [ ]:
GUID = !uuidgen
GUID = str(GUID).strip('[]')
!vdb-config --set LIBS/GUID={GUID}
!vdb-config --report-cloud-identity yes
!vdb-config --set /repository/user/main/public/cache-disabled=true
#!cat $HOME/.ncbi/user-settings.mkfg

In [ ]:
%%bash

cat cell_line_ER_HR.txt | xargs -P 12 -n 1 fastq-dump -X 1000000 -O cell_line_fastq

In [ ]:
%%bash

ls cell_line_fastq

In [ ]:
%%bash

datasets download genome accession GCF_000001405.39 --chromosomes 17 --include-gtf --exclude-protein --exclude-rna



In [ ]:
%%bash

unzip -o ncbi_dataset.zip -d GRCh38.p13

In [ ]:
%%bash

makeblastdb -in GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna -dbtype nucl -parse_seqids -out GRCh38.p13/GRCh38.p13



In [ ]:
%%bash

mkdir cell_line_bams

### Making alignments

This script can be automated, but for the purposes of this demo, we are choosing only two of the fastq files to align to Chromosome 17 - a hot spot of activity for HER genes.  

* SRR12060806 is HR+

* SRR12060824 is ER-/PR+

Doing a little more digging into bioproject PRJNA640820: "62 breast cell lines composed of 27 Triple negative breast cancers (TNBC), 5 Non malignant (NM) lines, 14 Hormone receptor positive (HR+) and 16 Her2 amplified lines (Her2amp) were profiled." This was a relatively recent dataset produced on a NextSeq here in Boston.

<img src="./img/citation.png" align="left" width="700"/>


https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE152908



In [ ]:
%%bash

magicblast -query cell_line_fastq/SRR12060806.fastq -db GRCh38.p13/GRCh38.p13 \
-splice T -no_unaligned -infmt fastq -num_threads 8 | \
samtools view -bS | samtools sort -@ 8 -o cell_line_bams/SRR12060806.bam -O BAM

In [ ]:
%%bash

magicblast -query cell_line_fastq/SRR12060824.fastq -db GRCh38.p13/GRCh38.p13 \
-splice T -no_unaligned -infmt fastq -num_threads 8 | \
samtools view -bS | samtools sort -@ 8 -o cell_line_bams/SRR12060824.bam -O BAM

In [ ]:
%%bash

samtools index cell_line_bams/SRR12060824.bam
samtools index cell_line_bams/SRR12060806.bam

## Load the Data in the NCBI Genome Data Viewer

These bam files with their indexes need to be loaded somewhere they can be retrieved by the NCBI Genome Data Viewer.

If you want to load your own remote files into GDV Viewer, it is possible using your Github repository for files that are **less than 1 GB**. Just use the following file format based on the location of your file in the repository.


Follow this format:  https:///<i></i>github.com/YOUR-GITHUB-ACCOUNT/YOUR-GITHUB-REPOSITORY/raw/master/PATH-TO-BAM



## See the Two Bams from this Demo

The two aligned bams have been pre-loaded at the following link: 
    

https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?cfg=NCID_1_53497080_130.14.18.128_9146_1631700092_3532671673

which will be good for the next 90 days.






Here are the links to the two bams if you want to load them again, make sure to zoom to Chromosome 17, that is where the reads were aligned. The screenshot below indicates where to enter these URL's to have the tracks show up.


<img src="./img/add_remote_view.png" align="left" width="600"/>


https://github.com/ncbi/ASHG-Workshop-2021/raw/main/bams/SRR12060806.bam


https://github.com/ncbi/ASHG-Workshop-2021/raw/main/bams/SRR12060824.bam

## What did we find???

Actually, starting with a general idea and targeting a specific question but being general in our search terms, this demo shows the power of using SQL to rapidly search millions of records in SRA. 

The speed and specificity of using SQL for will help researchers quickly locate sequences of interest, bring these sequences directly into a workflow and compare the outcomes within the command line interface.



***

#### The GDV link leads to the TP53 gene, an important protein in the development of cancer.



<img src="./img/tp53_gdv.png" align="left" width="600"/>



***

#### The mismatch in the pileups help narrow the scope of where interesting SNPs might be.



<img src="./img/pileup_view.png" align="left" width="600"/>



***

#### One of the cell lines has a SNP that indicates that cisplatin will be a less effective drug treatment.




<img src="./img/drug_response_variant.png" align="left" width="600"/>






***

#### Reading more into the records shows the importance of this variant for many cancer drugs.

<img src="./img/clinvar.png" align="left" width="600"/>

The cell line with the SNP may be a good candidate for drug therapy studies to emulate individuals whose cancers do not respond to cisplatin. 

# Thank you!!!

## Please Take our Survey!

https://nlmenterprise.co1.qualtrics.com/jfe/form/SV_bx6evo2ghfCjkrk
    
    